In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
"SET THE WORKING DIRECTORY BELOW TO THE LOCATION OF DATA FILES"

working_directory = 'C:/Users/conno/OneDrive/University Study/Honours Thesis/cnolan-thesis' #set location using back slashes

os.chdir(working_directory)

print("Current working directory: {0}".format(os.getcwd()))


def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
            output_path = os.makedirs(directory)
            print(output_path)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        

# Folder where outputs will be saved (by default a folder within the working directory) 
createFolder('./output/') 
output_path = working_directory +'./output/'

print('Set WD: Done')

# Import Data

In [ ]:
nger_data_matched = pd.read_csv('https://raw.githubusercontent.com/connorpn/cnolan-thesis/main/output/nger_data_matched.csv',encoding = "ISO-8859-1")
ms_data = pd.read_csv ('https://raw.githubusercontent.com/connorpn/cnolan-thesis/main/output/ms_data.csv', encoding='latin1')
ds_industry = pd.read_csv ('https://raw.githubusercontent.com/connorpn/cnolan-thesis/main/data/ds_ticker_industry.csv')

# Combine Matched NGER, Morningstar, and Datasets 

In [ ]:
emissions_determinants_data = pd.merge(ms_data[['ticker', 'morningstar_name', 'year', 'capex', 'eoy_price', 'eps', 'ltdebt', 'marketcap', 'ppe', 'revenue', 'roe', 'stdebt', 'assets', 'liabilities']],
                                  nger_data_matched[['year', 'ticker','matched', 'scope1', 'scope2', 'energy_consumption', 'total_emissions']],
                                  on = ['ticker','year'],
                                  how = 'left')

emissions_determinants_data = pd.merge(emissions_determinants_data,
                                       ds_industry,
                                      on=['ticker'],
                                      how='left')


emissions_determinants_data = emissions_determinants_data.reindex(columns=['year','ticker','morningstar_name','industry','matched', 'scope1', 'scope2', 'energy_consumption', 'total_emissions', 'capex', 'eoy_price', 'eps', 'ltdebt', 'marketcap', 'ppe', 'revenue', 'roe', 'stdebt', 'assets', 'liabilities'])



# Construct Dependent Variables (Carbon Emissions)

In [ ]:
"Construct Log(emissions)"


#log scope1, scope2, total_emissions, and energy consumption
emissions_determinants_data['log_scope1'] = np.log(emissions_determinants_data['scope1'])
emissions_determinants_data['log_scope2'] = np.log(emissions_determinants_data['scope2'])
emissions_determinants_data['log_total_emissions'] = np.log(emissions_determinants_data['total_emissions'])
emissions_determinants_data['log_energy_consumption'] = np.log(emissions_determinants_data['energy_consumption'])


"Construct year by year change in emissions"

#get a list of unique company names
unique_firms_emissions_determinants_data = (emissions_determinants_data['ticker'].unique())

#clone each unique company for each observation year (2009-2021)
firms_2009 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2009'})
firms_2010 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2010'})
firms_2011 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2011'})
firms_2012 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2012'})
firms_2013 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2013'})
firms_2014 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2014'})
firms_2015 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2015'})
firms_2016 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2016'})
firms_2017 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2017'})
firms_2018 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2018'})
firms_2019 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2019'})
firms_2020 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2020'})
firms_2021 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2021'})

#create dataframe to add all cloned firm years
firms_allyears = pd.DataFrame(columns = ['ticker', 'year'])

#add all clone firm year dataframes to a list
firm_years_list = [firms_2009, firms_2010, firms_2011, firms_2012, firms_2013, firms_2014, firms_2015, firms_2016, firms_2017, firms_2018, firms_2019, firms_2020, firms_2021]

#concatenate dataframes
firms_allyears = pd.concat(firm_years_list)

#convert year column to date time format
firms_allyears['year'] =  pd.to_datetime(firms_allyears['year'], format='%Y').dt.to_period("Y")
firms_allyears['year'] = firms_allyears['year'].astype(str)
#create dummy level index
emissions_determinants_data["dummy_index"] = emissions_determinants_data["dummy_index"] = 1
firms_allyears['dummy_index'] = firms_allyears['dummy_index'] = 2

#concenate cloned firm years with main data file
emissions_determinants_data = pd.concat([emissions_determinants_data, firms_allyears])

#sort dataframe by corporation name and year
emissions_determinants_data = emissions_determinants_data.sort_values(by=['ticker', 'year','dummy_index'])

#drop duplicates keeping first row (main  data file) as by dummy_index sorting
emissions_determinants_data = emissions_determinants_data.drop_duplicates(['ticker', 'year'], keep='first')

#sort dataframe by ticker and year
emissions_determinants_data = emissions_determinants_data.sort_values(by=['ticker', 'year'])

#calculate yearly change in emissions by corporation for scope1, scope2, total_emissions, and energy consumption
emissions_determinants_data['change_scope1'] = emissions_determinants_data.groupby(['ticker'])['scope1'].diff()
emissions_determinants_data['change_scope2'] = emissions_determinants_data.groupby(['ticker'])['scope2'].diff()
emissions_determinants_data['change_total_emissions'] = emissions_determinants_data.groupby(['ticker'])['total_emissions'].diff()
emissions_determinants_data['change_energy_consumption'] = emissions_determinants_data.groupby(['ticker'])['energy_consumption'].diff()


"Construct Intensity (Emissions)"

## int = (tons CO 2 e/AUD m.)
emissions_determinants_data['revenue(m)'] = emissions_determinants_data['revenue'] / 1000000
emissions_determinants_data['scope1_int'] = emissions_determinants_data['scope1'] /  emissions_determinants_data['revenue(m)']
emissions_determinants_data['scope2_int'] = emissions_determinants_data['scope2'] /  emissions_determinants_data['revenue(m)']
emissions_determinants_data['total_emissions_int'] = emissions_determinants_data['total_emissions'] /  emissions_determinants_data['revenue(m)']
emissions_determinants_data['energy_consumption_int'] = emissions_determinants_data['energy_consumption'] / emissions_determinants_data['revenue(m)']

# Construct Independent Variables

In [ ]:
#LOGSIZE
emissions_determinants_data['logsize'] = np.log(emissions_determinants_data['marketcap'])

#B/M
emissions_determinants_data['bm'] = (emissions_determinants_data['assets'] - emissions_determinants_data['liabilities']) / emissions_determinants_data['marketcap']

#ROE (already exisits 'roe')

#LEVERAGE
emissions_determinants_data['totaldebt'] = emissions_determinants_data.stdebt.fillna(0) + emissions_determinants_data.ltdebt.fillna(0) #calculate total debt, skipping nan values (this means total debt can be constructed from ltdebt, stdebt, or both)
emissions_determinants_data['leverage'] = emissions_determinants_data.totaldebt / emissions_determinants_data.assets #calculate leverage

#INVEST/A
emissions_determinants_data['investa'] = emissions_determinants_data.capex / emissions_determinants_data.assets

#HHI
#HHI WILL NOT BE INCLUDED DUE TO ABSENCE OF SEGEMENTED BUSINESS REVENUES

#LOGPPE
emissions_determinants_data['logppe'] = np.log(emissions_determinants_data['ppe'])

#SALESGR
emissions_determinants_data = emissions_determinants_data.sort_values(by=['ticker', 'year']) #sort dataframe by ticker and year
emissions_determinants_data['revenue_change'] = emissions_determinants_data.groupby(['ticker'])['revenue'].diff() #calculate yearly change in revenue by firm
emissions_determinants_data['salesgr'] = emissions_determinants_data.revenue_change / emissions_determinants_data.marketcap #salesgr = change in annual revenue normailzed by marketcap

#EPSGR 
emissions_determinants_data = emissions_determinants_data.sort_values(by=['ticker', 'year']) #sort dataframe by ticker and year
emissions_determinants_data['eps_change'] = emissions_determinants_data.groupby(['ticker'])['eps'].diff() #calculate yearly change in eps by firm
emissions_determinants_data['epsgr'] = emissions_determinants_data.eps_change / emissions_determinants_data.eoy_price

# Filter and Process Dataset for Regression

In [ ]:
emissions_determinants_data = emissions_determinants_data.dropna(axis=0, how= 'any', subset=['matched']).reset_index(drop=True)
emissions_determinants_data['year'] = emissions_determinants_data['year'].astype(int)
emissions_determinants_data = emissions_determinants_data.loc[(emissions_determinants_data['year'] >= 2009) & (emissions_determinants_data['year'] <= 2021)].reset_index(drop=True)
emissions_determinants_data = emissions_determinants_data.sort_values(by=['ticker', 'year']).reset_index(drop=True)

emissions_determinants_vars = emissions_determinants_data[['year', 'ticker','industry','log_scope1', 'log_scope2', 'log_total_emissions', 'log_energy_consumption', 'change_scope1', 'change_scope2', 'change_total_emissions', 'change_energy_consumption', 'scope1_int', 'scope2_int', 'total_emissions_int', 'energy_consumption_int', 'logsize', 'bm', 'roe','leverage', 'investa', 'logppe', 'salesgr', 'epsgr']]
emissions_determinants_vars = emissions_determinants_vars.sort_values(by=['year','ticker']).reset_index(drop=True)
emissions_determinants_vars = emissions_determinants_vars.dropna(how='all', subset=['log_scope1', 'log_scope2', 'log_total_emissions', 'log_energy_consumption', 'change_scope1', 'change_scope2', 'change_total_emissions', 'change_energy_consumption', 'scope1_int', 'scope2_int', 'total_emissions_int', 'energy_consumption_int']).reset_index(drop=True)
emissions_determinants_vars = emissions_determinants_vars.dropna(how='any', subset=['logsize', 'bm', 'roe', 'leverage', 'investa', 'logppe', 'salesgr', 'epsgr','industry']).reset_index(drop=True)

"Save Determinants of Carbon Emissions Variables Dataset (for regression use)"
output_filename = 'emissions_determinants_vars.csv'
outputname = output_path + output_filename
emissions_determinants_vars.to_csv(outputname, mode='w',index=False)
print("Exported File: " + outputname)

# Constructed Variables for The Determinants of Carbon Emissions

In [ ]:
print('Number of Observations:')
print (len(emissions_determinants_vars))

with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
    display(emissions_determinants_vars)

In [ ]:
print('Notebook Finish')